In [1]:
import pickle
import pandas as pd

ratings = pickle.load( open( "saveKidsAndAdultsM.p", "rb" ) )


In [2]:
ratingsPD = pd.DataFrame(ratings, columns=['user', 'item', 'rating'] )
ratingsPD.head()

,user,item,rating
0,0,45,33.0
1,0,53,3.0
2,0,78,128.0
3,0,249,4.0
4,0,283,12.0


In [3]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
scaler = preprocessing.MinMaxScaler([1,5])
def fx(x):
    data = preprocessing.scale(x['rating'].to_numpy().reshape(-1, 1))
    scaler.fit(data)
    data=scaler.transform(data)
    x['rating']=data
    return x
ratingsPD = ratingsPD.groupby('user').apply(fx)

In [4]:
ratingsPD1= ratingsPD.groupby('user').filter(lambda x : len(x)>24)


In [5]:
ratingsPD2 = ratingsPD1.groupby('item').filter(lambda x : len(x)>3)

In [6]:
ratings =ratingsPD2

In [7]:
len(ratings)

10558921

In [8]:
ratings.groupby('user').count()

,item,rating
user,,
0,864,864
1,1936,1936
2,798,798
3,469,469
4,1425,1425
5,887,887
6,1757,1757
7,215,215
8,836,836


In [9]:
ratings['item'].nunique()

232070

In [10]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias
rm = 0
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5,xf.SampleFrac(0.3)):
    pred = als.BiasedMF(45)
    #rec = Recommender.adapt(pred)
    pred.fit(train)
    preds = batch.predict(pred, test)
    #print(preds['prediction'].isna().sum())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))
    rm+= (rmse(preds['prediction'], preds['rating'],missing ='ignore'))
print("Average RMSE: " + str(rm/5))


0.34728041962070677
0.34333458618167584
0.34426389373906613
0.34575997515723317
0.34490501519125194
Average RMSE: 0.3451087779779868


In [ ]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias
rm = 0
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5,xf.SampleFrac(0.3)):
    pred = als.ImplicitMF(45)
    #rec = Recommender.adapt(pred)
    pred.fit(train)
    preds = batch.predict(pred, test)
    #print(preds['prediction'].isna().sum())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))
    rm+= (rmse(preds['prediction'], preds['rating'],missing ='ignore'))
print("Average RMSE: " + str(rm/5))